# Carregar dados

In [24]:
import pandas as pd

# # Mostrar mais colunas
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)

df = pd.read_parquet("./output/silver/rais_TODOS_ANOS_RAIS_VINC_PUB.parquet.gzip")


qnt_full = len(df)
print(f"""
Quantidade docs: {qnt_full}
""")

df.head(2)



Quantidade docs: 384175



,ano,id_uf_municipio,remuneracao_media,desligamento_id,idade,ocupacao_id,sexo_id,raca_cor_id,sigla_uf,sigla_municipio
0,2019,500240,997.99,12,17,317210,1,2,MS,Caarapo
1,2019,500570,0.00,11,36,212420,1,2,MS,Navirai


In [25]:
df_raca_cor = pd.read_csv("./output/bronze/planilhas/dic_raca_cor.csv")
df_sexo = pd.read_csv("./output/bronze/planilhas/dic_sexo.csv")
df_ordered_sinonimo = pd.read_csv("./output/bronze/planilhas/ordered_sinonimo.csv")

In [26]:
# change column ocupacao_id to int
df["ocupacao_id"] = df["ocupacao_id"].astype(int)

# Juntando tabelas

In [27]:
# merge df and df_sexo by sexo_id
df = pd.merge(df, df_raca_cor, left_on="raca_cor_id",
              right_on="chave", how="left")
df = pd.merge(df, df_sexo, left_on="sexo_id", right_on="chave",
              how="left")
df = pd.merge(df, df_ordered_sinonimo, left_on="ocupacao_id",
              right_on="cbo_2002", how="left")
df.head(2)

,ano,id_uf_municipio,remuneracao_media,desligamento_id,idade,ocupacao_id,sexo_id,raca_cor_id,sigla_uf,sigla_municipio,chave_x,valor_x,chave_y,valor_y,cbo_2002,sinonimo
0,2019,500240,997.99,12,17,317210,1,2,MS,Caarapo,2.0,Branca,1,Masculino,317210.0,Analista de service desk
1,2019,500570,0.00,11,36,212420,1,2,MS,Navirai,2.0,Branca,1,Masculino,212420.0,Analista de suporte


# Gerando ID para Sigla_UF

In [28]:
# Create id for sigla_uf
df["sigla_uf_id"] = df["sigla_uf"].astype("category").cat.codes
df.head(2)


,ano,id_uf_municipio,remuneracao_media,desligamento_id,idade,ocupacao_id,sexo_id,raca_cor_id,sigla_uf,sigla_municipio,chave_x,valor_x,chave_y,valor_y,cbo_2002,sinonimo,sigla_uf_id
0,2019,500240,997.99,12,17,317210,1,2,MS,Caarapo,2.0,Branca,1,Masculino,317210.0,Analista de service desk,11
1,2019,500570,0.00,11,36,212420,1,2,MS,Navirai,2.0,Branca,1,Masculino,212420.0,Analista de suporte,11


# Desligamento

In [30]:
# Create new col desligamento from desligament_id, if != 0, so "nao", otherwise "sim"
df["desligamento"] = df["desligamento_id"].apply(
    lambda x: 0 if x != 0 else 1)
df.head(2)



,ano,id_uf_municipio,remuneracao_media,desligamento_id,idade,ocupacao_id,sexo_id,raca_cor_id,sigla_uf,sigla_municipio,chave_x,valor_x,chave_y,valor_y,cbo_2002,sinonimo,sigla_uf_id,desligamento
0,2019,500240,997.99,12,17,317210,1,2,MS,Caarapo,2.0,Branca,1,Masculino,317210.0,Analista de service desk,11,0
1,2019,500570,0.00,11,36,212420,1,2,MS,Navirai,2.0,Branca,1,Masculino,212420.0,Analista de suporte,11,0


# Selecionando colunas necessárias

In [31]:
df = df[["ano", "sigla_uf_id", "sigla_uf", "remuneracao_media", 'desligamento', "idade",
         "ocupacao_id", "sinonimo", "sexo_id", "valor_y", "raca_cor_id", "valor_x"]]

df = df.rename(columns={
    "sinonimo": "ocupacao",
    "valor_y": "sexo", 
    "valor_x": "raca_cor"
    })

df.head(2)

,ano,sigla_uf_id,sigla_uf,remuneracao_media,desligamento,idade,ocupacao_id,ocupacao,sexo_id,sexo,raca_cor_id,raca_cor
0,2019,11,MS,997.99,0,17,317210,Analista de service desk,1,Masculino,2,Branca
1,2019,11,MS,0.00,0,36,212420,Analista de suporte,1,Masculino,2,Branca


# Salvando arquivo

In [32]:
df.to_parquet("./output/gold/rais_TODOS_ANOS_comJoin_RAIS_VINC_PUB.parquet.gzip", index=False, compression="gzip")